In [1]:
from TM1py.Services import TM1Service
import configparser
import datetime
import pandas as pd

# Connect to DEV Server - CAM or Keyring for PROD!!
config = configparser.ConfigParser()
config.read(r'./import/config.ini')
tm1 = TM1Service(**config['csvimport'])

try:
    server_name = tm1.server.get_server_name()
    print("Connection to TM1 established!! your Servername is: {}".format(server_name))
except Exception as e:
    print("\nERROR:")
    print("\t" + str(e))
    
# Log lesen und in zwei DataFrames speichern
def parse_tm1_log():
    dt = datetime.date.today() - datetime.timedelta(days=2)
    current_date = datetime.datetime.combine(dt, datetime.datetime.min.time())
    #print(current_date)
    log_entries = tm1.server.get_message_log_entries(since = current_date)
    info_data = []
    error_data = []

    for entry in log_entries:
        # {'ID': 12636, 'ThreadID': 10196, 'SessionID': 0, 'Level': 'Info', 'TimeStamp': '2024-01-29T19:50:50.908Z', 'Logger': 'TM1.Server', 'Message': 'Admin-Server wird benachrichtigt'}
        log_level = entry['Level']
        logger = entry['Logger']
        message = entry['Message']
        timestamp = entry['TimeStamp']

        if log_level == 'Info':
            info_data.append({'Timestamp': timestamp, 'Logger': logger, 'Message': message})
        elif log_level == 'Error':
            error_data.append({'Timestamp': timestamp, 'Logger': logger,  'Message': message})

    info_df = pd.DataFrame(info_data)
    error_df = pd.DataFrame(error_data)
    
    return info_df, error_df

# Exportieren der DataFrames in CSV
def export_to_csv(df, file_name):
    df.to_csv(file_name, sep=';', index=False, encoding='utf-8')
    print("Check updated in {}".format(file_name))

# Parse TM1-Log und Export DF
infos, errors = parse_tm1_log()
export_to_csv(errors, 'exports/Logging_' + server_name + '_errors.csv')
export_to_csv(infos, 'exports/Logging_'+ server_name + '_infos.csv')

Connection to TM1 established!! your Servername is: csvimport
Check updated in exports/Logging_csvimport_errors.csv
Check updated in exports/Logging_csvimport_infos.csv
